In [91]:
# !pip install transformers fugashi[unidic-lite] ipadic
!pip list | grep 'transformers\|fugashi\|ipadic\|tensorflow'

fugashi                        1.1.0
ipadic                         1.0.0
tensorflow                     2.4.1
tensorflow-addons              0.12.1
tensorflow-cloud               0.1.13
tensorflow-data-validation     0.29.0
tensorflow-datasets            3.0.0
tensorflow-estimator           2.4.0
tensorflow-gpu                 2.4.1
tensorflow-hub                 0.9.0
tensorflow-metadata            0.29.0
tensorflow-model-analysis      0.29.0
tensorflow-serving-api         2.4.1
tensorflow-transform           0.29.0
transformers                   4.8.1


In [11]:
from transformers import AutoTokenizer

checkpoint = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [92]:
raw_inputs = [
    #'テレビでサッカーの試合を見る。', 
    'テレビで' + tokenizer.mask_token + 'の試合を見る。', 
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[   2,  571,   12,    4,    5,  608,   11, 2867,    8,    3]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


In [93]:
from transformers import TFAutoModelForMaskedLM

model = TFAutoModelForMaskedLM.from_pretrained(checkpoint)
outputs = model(inputs)
print(outputs.logits.shape)

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


(1, 10, 32000)


In [94]:
import tensorflow as tf

predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[[3.7459305e-08 1.0108354e-03 3.0044521e-07 ... 9.5772705e-08
   1.4990446e-08 1.3698190e-07]
  [2.2006600e-13 5.4908986e-07 3.1778032e-11 ... 6.3342663e-13
   2.9647142e-13 1.1226209e-12]
  [1.6911761e-15 4.8180787e-08 1.7189679e-12 ... 1.9864927e-14
   4.0749305e-14 4.7688399e-14]
  ...
  [3.1071811e-13 1.9899795e-07 2.3371231e-12 ... 3.6241833e-14
   6.2069416e-12 2.8103923e-14]
  [1.0250574e-12 2.7231323e-07 6.7588213e-10 ... 3.3563010e-11
   5.8855171e-11 1.3667138e-12]
  [3.8539120e-12 5.3201114e-07 7.5232398e-10 ... 1.7161642e-10
   6.5043609e-12 1.3238502e-12]]], shape=(1, 10, 32000), dtype=float32)


In [95]:
results = tf.math.top_k(tf.math.softmax(outputs.logits[0, 3, :], axis=-1), k=3)  # get top 3 candidates
results.indices.numpy()

array([ 1301,  5653, 15235], dtype=int32)

In [96]:
# Show results
for i, index in enumerate(results.indices):
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

0 サッカー
1 バスケットボール
2 阪神タイガース


In [72]:
unmasker = pipeline("fill-mask", model="cl-tohoku/bert-base-japanese-whole-word-masking", tokenizer='cl-tohoku/bert-base-japanese-whole-word-masking')

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [79]:
unmasker("テレビで" + unmasker.tokenizer.mask_token + "の試合を見る。", top_k=3)

[{'sequence': 'テレビ で サッカー の 試合 を 見る 。',
  'score': 0.08827631920576096,
  'token': 1301,
  'token_str': 'サ ッ カ ー'},
 {'sequence': 'テレビ で バスケットボール の 試合 を 見る 。',
  'score': 0.057113878428936005,
  'token': 5653,
  'token_str': 'バ ス ケ ッ ト ボ ー ル'},
 {'sequence': 'テレビ で 阪神タイガース の 試合 を 見る 。',
  'score': 0.04361201822757721,
  'token': 15235,
  'token_str': '阪 神 タ イ ガ ー ス'}]

In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9598047137260437}]

In [2]:
classifier([
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!"
])

[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]